In [24]:
import tensorflow as tf
import pandas as pd
import os
from sklearn.model_selection import train_test_split

In [25]:
# returns dataframe
# params: full -> True = full dataset, False = first file

def grab_data(full, path):
    if full:
        return pd.concat([pd.read_csv(path+file, index_col=False) for file in os.listdir(path)])
    else:
        for file in os.listdir(path):
            return pd.read_csv(path+file, index_col=False)

In [26]:
# import data

nBodies = 3
workDir = "C:/Users/llave/Documents/nBody/"
dataPath = workDir + "/data/"
        
df = grab_data(True, dataPath)
df.head()

,finalFile,eventID,m1,m2,m3,x1,x2,x3,y1,y2,...,xf3,yf1,yf2,yf3,dxf1,dxf2,dxf3,dyf1,dyf2,dyf3
0,1,10001,84.270385,56.314115,52.279696,-2.226488,-3.406637,9.440128,-3.887835,-4.902674,...,9.442821,-3.888589,-4.899092,-2.849904,0.457428,-0.525155,0.687588,-0.222422,0.961199,-0.100621
1,1,10002,84.270385,56.314115,52.279696,-2.226488,-3.406637,9.440128,-3.887835,-4.902674,...,9.445500,-3.889573,-4.895165,-2.850298,0.389737,-0.420452,0.683916,-0.281080,1.049364,-0.101038
2,1,10003,84.270385,56.314115,52.279696,-2.226488,-3.406637,9.440128,-3.887835,-4.902674,...,9.448164,-3.890785,-4.890894,-2.850693,0.321736,-0.315285,0.680246,-0.339542,1.137235,-0.101454
3,1,10004,84.270385,56.314115,52.279696,-2.226488,-3.406637,9.440128,-3.887835,-4.902674,...,9.450814,-3.892225,-4.886281,-2.851090,0.253345,-0.209536,0.676576,-0.397869,1.224904,-0.101870
4,1,10005,84.270385,56.314115,52.279696,-2.226488,-3.406637,9.440128,-3.887835,-4.902674,...,9.453450,-3.893893,-4.881325,-2.851489,0.184487,-0.103088,0.672908,-0.456125,1.312466,-0.102286


In [34]:
# INPUT: want mass, x/y pos, dx/dy for n bodies -> total input is n*5 + 1 (time)
# OUTPUT: want x/y, dx/dy -> total input is n*4

i_col, o_col = [], []
colNames = ["m", "x", "y", "dx", "dy"]

for col in colNames:
    for n in range(1, nBodies+1):
        i_col.append(col+str(n))
        if col != "m":
            o_col.append(col+"f"+str(n))
i_col.append("t")
    
X1 = df[i_col].values
y1 = df[o_col].values

X_train,X_test,y_train,y_test = train_test_split(X1, y1, test_size=0.2, random_state=42)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(1147503, 16) (1147503, 12)
(286876, 16) (286876, 12)


In [35]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(X_train.shape[1], activation='relu', input_dim=X_train.shape[1]),
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dense(y_train.shape[1],activation='linear')
])

loss_fn = tf.keras.losses.MeanSquaredError()
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['mse'])

In [36]:
model.fit(X_train,y_train, epochs=5)

Train on 1147503 samples
Epoch 1/5
1147503/1147503 [==============================] - 57s 49us/sample - loss: 6.7853 - mse: 6.7853
Epoch 2/5
1147503/1147503 [==============================] - 60s 52us/sample - loss: 4.3517 - mse: 4.3517
Epoch 3/5
1147503/1147503 [==============================] - 63s 55us/sample - loss: 4.0187 - mse: 4.0187
Epoch 4/5
1147503/1147503 [==============================] - 74s 64us/sample - loss: 3.9256 - mse: 3.9256
Epoch 5/5
1147503/1147503 [==============================] - 71s 62us/sample - loss: 3.8403 - mse: 3.8403


In [37]:
model.evaluate(X_test,  y_test, verbose=2)

286876/286876 - 5s - loss: 3.7868 - mse: 3.7868


[3.786778229787759, 3.7867806]